In [6]:
import pandas as pd
import json
import os
from stringUtils import match_entities

In [9]:
import sys
print sys.path

['', '/usr/local/lib/python2.7/dist-packages/kenlm-0.0.0-py2.7-linux-x86_64.egg', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/usr/local/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages/gtk-2.0', '/usr/local/lib/python2.7/dist-packages/IPython/extensions', '/home/xinyi/.ipython']


In [10]:
INPUT_PREFIX = "../../datasets/ComplexWebQuestions_1_1/Data/"

In [11]:
import ast
import re
def read_mids_df(path):
    mids = pd.read_csv(path, encoding='utf-8')
    mids['names'] = mids['names'].fillna('[]')
    mids['names'] = mids['names'].apply(lambda x: str(x).replace('][', ','))
    mids['names'] = mids['names'].apply(ast.literal_eval)
    return mids

In [12]:
from stringUtils import fuzzy_match_entities
from stringUtils import sim_match_entities
def process_split_part1(row, mid_df):
    mid_df_sub = mid_df[mid_df['ID'] == row.ID]
    mid_df_sub = mid_df_sub.drop_duplicates('mid')
    new_split_part = row.split_part1.copy()
    sub_ques = new_split_part[u"sub_ques"]
    mid_df_sub = mid_df[mid_df['ID'] == row.ID]
    mid_df_sub = mid_df_sub.drop_duplicates('mid')
    matched = []
    for idx, mid_row in mid_df_sub.iterrows():
        mid = mid_row.mid
        if mid is None:
            continue
        names = mid_row.names
        found = 0
        for name in names:
            m = re.search(re.escape(name), sub_ques, re.IGNORECASE)
            if m:
                matched.append({"mid": mid, "mention": name})
                found = 1
                break
        if found == 0:
            matched_seq, score = sim_match_entities(sub_ques, names)
            if matched_seq:
                print "Question: " + row.question
                print "Matched Sequence: " + matched_seq
                print "Score: " + str(score)
                print '\n'
                matched.append({"mid":mid, "mention": matched_seq})
    return {"sub_ID":new_split_part["sub_ID"], "sub_ques": sub_ques, "entities": matched}

def process_split_part2(row, mid_df):
    mid_df_sub = mid_df[mid_df['ID'] == row.ID]
    mid_df_sub = mid_df_sub.drop_duplicates('mid')
    new_split_part = row.split_part2.copy()
    sub_ques = new_split_part[u"sub_ques"]
    mid_df_sub = mid_df[mid_df['ID'] == row.ID]
    mid_df_sub = mid_df_sub.drop_duplicates('mid')
    matched = []
    for idx, mid_row in mid_df_sub.iterrows():
        mid = mid_row.mid
        if mid is None:
            continue
        names = mid_row.names
        found = 0
        for name in names:
            m = re.search(re.escape(name), sub_ques, re.IGNORECASE)
            if m:
                matched.append({"mid": mid, "mention": name})
                found = 1
                break
        if found == 0:
            matched_seq, score = sim_match_entities(sub_ques, names)
            if matched_seq:
                print "Question: " + row.question
                print "Matched Sequence: " + matched_seq
                print "Score: " + str(score)
                print '\n'
                matched.append({"mid":mid, "mention": matched_seq})
    return {"sub_ID":new_split_part["sub_ID"], "sub_ques": sub_ques, "entities": matched}

In [13]:
choices = ['dev']
for choice in choices:
    print "start " + choice
    file_path = os.path.join(INPUT_PREFIX, "annotated_complex_web_questions_bk/" + choice + ".json")
    data = json.load(open(file_path))
    questions = pd.DataFrame(data)
    mid_df = read_mids_df(os.path.join(INPUT_PREFIX, 'qid_mid_names/' + choice + '_mid_names.tsv'))
    questions["new_split_part1"] = questions.apply(process_split_part1, args=(mid_df,), axis=1)
    questions["new_split_part2"] = questions.apply(process_split_part2, args=(mid_df,), axis=1)
    questions["split_part1"] = questions["new_split_part1"]
    questions["split_part2"] = questions["new_split_part2"]
    questions = questions.drop(['new_split_part1', 'new_split_part2'], axis=1)
    with open(os.path.join(INPUT_PREFIX, 'annotated_complex_web_questions/' + choice + ".json"), 'w+') as output_file:
        json.dump(questions.to_dict(orient='records'), output_file, indent=4)

start dev
Question: What is the mascot of the team that has Nicholas S. Zeppos as its leader?
Matched Sequence: nicholas s zeppos
Score: 0.482142857143


Question: Which country that borders Skovakia is Kiev the capital of?
Matched Sequence: skovakia
Score: 0.5


Question: Which city hosted the 1906 Intercalted games and was knows for having Greek Temples?
Matched Sequence: 1906 intercalted games
Score: 0.406976744186


Question: From where is the University of South Carolina Beaufort Child Organization?
Matched Sequence: university of south carolina beaufort
Score: 0.521739130435


Question: What kind of money does the country with the nation antem Du gamla, Du fria use?
Matched Sequence: du gamla , du fria
Score: 0.45


Question: The country that contains Balochistan, Pakistan had what President in 1980?
Matched Sequence: balochistan
Score: 0.5


Question: The SacatepÃ©quez Department is which of 3 countries bordering Mexico?
Matched Sequence: sacatepaquez department
Score: 0.5061728

Question: What country is the birthplace of the person who said, "A good GI bill would increase the recruit pool"?
Matched Sequence: good gi bill would increase the recruit pool
Score: 0.780141843972


Question: What countries does the person who said "He speaks to Me as if I was a public meeting" rule?
Matched Sequence: he speaks to me as if i was a public meeting
Score: 0.707142857143


Question: Which peer of Friedrich HÃ¶lderlin inspired the work of Walt Whitman?
Matched Sequence: friedrich halderlin
Score: 0.538461538462


Question: What art movement practiced by the person that said "The best way to know God is to love many things."?
Matched Sequence: the best way to know god is to love many things
Score: 0.812949640288


Question: What University did the author who published editions for, That was then, this is now, attend?
Matched Sequence: that was then , this is
Score: 0.605263157895


Question: On which continent is the Old Israeli sheckel accepted as currency?
Matched Seque

Question: WHat three countries border the nation where the Purepecha langiage is spoken?
Matched Sequence: purepecha langiage
Score: 0.43661971831


Question: Who wrote the Story of Glittering Plain that inspired Antoni Gaudi?
Matched Sequence: the story of glittering plain
Score: 0.456896551724


Question: Who is rich? He that rejoices in his portion, was said by a person with what job?
Matched Sequence: who is rich he that rejoices in his portion
Score: 0.742647058824


Question: A state held the 2008 United States Senate Election in Tennessee; what is its governor?
Matched Sequence: 2008 united states senate election in tennessee
Score: 0.536144578313


Question: What European Union country's government is called the government of Ireland?
Matched Sequence: government
Score: 0.466666666667


Question: Who influenced the poetry of the person who said, "I accept reality and dare not question it"?
Matched Sequence: accept reality and dare not question it
Score: 0.931034482759


Questio

Question: Which Jonas brother ended a celebrity relationship in 2007?
Matched Sequence: jonas brother
Score: 0.4375


Question: What country bordering China appointed Emomali Rahmon to a governmental position?
Matched Sequence: emomali rahmon
Score: 0.403846153846


Question: Which countries does the Niger River flow through, where the location that appointed Dioncounda TraorÃ© to a governmental position?
Matched Sequence: dioncounda traora
Score: 0.508474576271


Question: What airport is close to Sandals Grande in the country with national anthem Sons and daughters of St. Lucia?
Matched Sequence: sons and daughters of st
Score: 0.427272727273


Question: Which state has Jim Edgar as a government member and includes the Ilinois River as a landmark?
Matched Sequence: ilinois river
Score: 0.4375


Question: What is the name of the main train station of the city that Diane Feinstein was appointed for a governmental position?
Matched Sequence: feinstein
Score: 0.416666666667


Question: W